In [1]:
import numpy as np
import keras
import matplotlib.pylab as plt
import pandas as pd
import string
import pydot
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Flatten, LSTM, Dropout, Activation, BatchNormalization, Bidirectional, GlobalMaxPool1D
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D,MaxPooling1D
from keras.models import Sequential
from keras.regularizers import l2
from sklearn.model_selection import train_test_split
#from create_frame import alldata,boolarr
from sklearn.metrics import classification_report, confusion_matrix
from keras import backend as K
from keras.utils import plot_model

C:\Users\Manasa\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
transcripts=pd.read_json('NETFLIX_EARRNINGS_CALL_TRANSCRIPT_Q3_2018.json')
#data = pd.read_json('NETFLIX_Q3_2018_EARNINGS_CALL_TRANSCRIPT_FANG.json')
data = pd.DataFrame.sort_index(transcripts)
x = data.text
#x = list(x)
y = data.Sentiments
y = y.values

In [3]:
#X, y = (data['text'].values, data['Sentiments'].values)

In [4]:
Y = y

In [5]:
for n,i in enumerate(Y):
    if i == 'Neutral' :
        Y[n] = 0
    elif i == 'Positive':
        Y[n] = 1
    else:
        Y[n] = -1

In [6]:
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, -1, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, -1, 0, -1, 0, 1, 0, 1, 0, 1, 0,
       -1, 0, 1, 0, 1, 1, -1, 0, 0, 0, -1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, -1, 1, 0, 0, 0, 1, 1, 1, 0, -1,
       0, 1, 0, 1, 0, 1, 0, 0, 0, -1, 0, 0, 0, -1, 0, -1, 0, -1, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, -1, 1, 1, 0,
       0, 1], dtype=object)

In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tk = Tokenizer(lower = True)
tk.fit_on_texts(x)
X_seq = tk.texts_to_sequences()
X_pad = pad_sequences(X_seq, maxlen=100, padding='post')

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size = 0.2, random_state = 1)

In [14]:
batch_size = 64
X_train1 = X_train[batch_size:]
y_train1 = y_train[batch_size:]
X_valid = X_train[:batch_size]
y_valid = y_train[:batch_size]

In [45]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
vocabulary_size = len(tk.word_counts.keys())+1
max_words = 100
embedding_size = 32
model = Sequential()

model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
#model.add(LSTM(200))
model.add(LSTM(256,return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))



#model.add(LSTM(200))



model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [40]:
vocabulary_size

1307

In [46]:

epochs=100

model.fit(X_train1, y_train1, epochs=epochs, batch_size=batch_size, shuffle=True)
scores = model.evaluate(X_test, y_test)
print(scores)

Epoch 1/100
76/76 [==============================] - 4s 58ms/step - loss: 0.6852 - acc: 0.5395
Epoch 2/100
76/76 [==============================] - 1s 16ms/step - loss: 0.8862 - acc: 0.5526
Epoch 3/100
76/76 [==============================] - 1s 17ms/step - loss: 0.6831 - acc: 0.5526
Epoch 4/100
76/76 [==============================] - 1s 16ms/step - loss: 0.6755 - acc: 0.5526
Epoch 5/100
76/76 [==============================] - 2s 20ms/step - loss: 0.6594 - acc: 0.5526
Epoch 6/100
76/76 [==============================] - 1s 17ms/step - loss: 0.6399 - acc: 0.5526
Epoch 7/100
76/76 [==============================] - 1s 16ms/step - loss: 0.6400 - acc: 0.5921
Epoch 8/100
76/76 [==============================] - 1s 16ms/step - loss: 0.6659 - acc: 0.6316
Epoch 9/100
76/76 [==============================] - 1s 16ms/step - loss: 0.6487 - acc: 0.6579
Epoch 10/100
76/76 [==============================] - 1s 16ms/step - loss: 0.5435 - acc: 0.6974
Epoch 11/100
76/76 [=============================

76/76 [==============================] - 1s 19ms/step - loss: 0.3048 - acc: 0.8947
Epoch 87/100
76/76 [==============================] - 1s 18ms/step - loss: 0.2999 - acc: 0.8947
Epoch 88/100
76/76 [==============================] - 1s 17ms/step - loss: 0.3134 - acc: 0.8947
Epoch 89/100
76/76 [==============================] - 1s 19ms/step - loss: 0.2977 - acc: 0.8947
Epoch 90/100
76/76 [==============================] - 1s 17ms/step - loss: 0.2868 - acc: 0.8947
Epoch 91/100
76/76 [==============================] - 1s 17ms/step - loss: 0.2791 - acc: 0.8947
Epoch 92/100
76/76 [==============================] - 1s 17ms/step - loss: 0.3033 - acc: 0.8947
Epoch 93/100
76/76 [==============================] - 1s 16ms/step - loss: 0.2942 - acc: 0.8947
Epoch 94/100
76/76 [==============================] - 1s 17ms/step - loss: 0.3008 - acc: 0.8947
Epoch 95/100
76/76 [==============================] - 1s 17ms/step - loss: 0.2988 - acc: 0.8947
Epoch 96/100
76/76 [==============================] -

In [47]:
#Confusion Matrix:
pred_test=model.predict(X_test)    

In [50]:
test_cm=np.int8(y_test)
pred_cm=np.int8(pred_test.round())

In [51]:
from sklearn.metrics import  confusion_matrix
cm=confusion_matrix(test_cm,pred_cm)
cm

array([[ 0,  0,  4],
       [ 0, 16,  6],
       [ 0,  0, 10]], dtype=int64)

In [54]:
#Training data confusion matrix
pred_train=model.predict(X_train)   



In [56]:
traint_cm=np.int8(y_train)
pred_train_cm=np.int8(pred_train.round())

In [57]:
cm_t=confusion_matrix(traint_cm,pred_train_cm)
cm_t

array([[ 0,  0,  9],
       [ 0, 61, 15],
       [ 0,  0, 55]], dtype=int64)